In [ ]:
import spacy
import pandas as pd
from collections import Counter
import re

# Load spaCy's small English model for NER
nlp = spacy.load("en_core_web_sm")

# Sample log data (this would typically be read from log files)
log_data = """
INFO: Connection from 192.168.1.1 established.
ERROR: Unable to open file /home/user/data.txt.
WARNING: Potential DDoS attack from IP 172.16.0.2 detected.
INFO: Connection closed from 192.168.1.1.
ERROR: Authentication failure from IP 192.168.1.1.
"""

# Process the log data using spaCy NLP
def process_logs(log_data):
    doc = nlp(log_data)
    
    # Extract entities from logs (IP addresses, file paths, etc.)
    ip_addresses = [ent.text for ent in doc.ents if ent.label_ == "IP"]
    error_messages = [sent.text for sent in doc.sents if "ERROR" in sent.text]
    
    # Count occurrences of each IP address (or other entities)
    ip_counts = Counter(ip_addresses)
    
    return ip_counts, error_messages

# Call the function to process the sample log data
ip_counts, error_messages = process_logs(log_data)
print("IP Address Counts:", ip_counts)
print("Error Messages:", error_messages)

In [ ]:
# Predefined list of suspicious IPs or known IOCs
suspicious_ips = ["192.168.1.1", "172.16.0.2"]

# Function to check for known threats in logs
def detect_threats(ip_counts, error_messages):
    # Flag to indicate if any threat is found
    threats = []
    
    for ip, count in ip_counts.items():
        if ip in suspicious_ips:
            threats.append(f"Suspicious IP {ip} found with {count} connections.")
    
    for message in error_messages:
        if "Authentication failure" in message:
            threats.append(f"Authentication failure detected: {message}")
    
    return threats

# Detect threats in the processed log data
threats = detect_threats(ip_counts, error_messages)
print("Detected Threats:", threats)

In [ ]:
# Function to search for a keyword in logs
def search_logs(keyword, log_data):
    matches = [line for line in log_data.split("\n") if keyword.lower() in line.lower()]
    return matches

# Search for "Authentication failure" in the logs
search_results = search_logs("Authentication failure", log_data)
print("Search Results:", search_results)

In [ ]:
from flask import Flask, render_template, request
import os

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/analyze", methods=["POST"])
def analyze():
    # Get the uploaded file
    uploaded_file = request.files["logfile"]
    if uploaded_file.filename != "":
        # Save the uploaded file
        file_path = os.path.join("uploads", uploaded_file.filename)
        uploaded_file.save(file_path)

        # Process the log file
        with open(file_path, "r") as f:
            log_data = f.read()

        ip_counts, error_messages = process_logs(log_data)
        threats = detect_threats(ip_counts, error_messages)

        return render_template("result.html", ip_counts=ip_counts, error_messages=error_messages, threats=threats)

    return "No file uploaded!"

if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Threat Hunting Assistant</title>
</head>
<body>
    <h1>Upload Log File for Analysis</h1>
    <form action="/analyze" method="post" enctype="multipart/form-data">
        <label for="logfile">Choose Log File:</label>
        <input type="file" name="logfile" required><br><br>
        <button type="submit">Analyze Logs</button>
    </form>
</body>
</html>

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Analysis Results</title>
</head>
<body>
    <h1>Analysis Results</h1>
    <h3>IP Address Counts</h3>
    <ul>
        {% for ip, count in ip_counts.items() %}
            <li>{{ ip }}: {{ count }}</li>
        {% endfor %}
    </ul>

    <h3>Error Messages</h3>
    <ul>
        {% for message in error_messages %}
            <li>{{ message }}</li>
        {% endfor %}
    </ul>

    <h3>Detected Threats</h3>
    <ul>
        {% for threat in threats %}
            <li>{{ threat }}</li>
        {% endfor %}
    </ul>
</body>
</html>